In [1]:
import os
import numpy as np
from scipy.sparse import linalg as LA
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from dataclasses import dataclass
from typing import List, Optional, Tuple

# —— 建议（可选）：避免过度并行（在导入 numpy/scipy 之前设置更佳）——
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")

# ===== 你已有的函数/类（原样使用，或用你自己的版本） =====

def bicut_group(L):
    """
    Enhanced spectral clustering function that returns both sign-based and optimal cuts.
    Args:
        L: numpy or scipy sparse (sub-)graph Laplacian (symmetric PSD)
    Returns:
        tuple: (first_group, second_group) (local indices)
    """
    n = L.shape[0]
    if n == 0:
        raise ValueError("The Laplacian matrix is empty.")
    if n == 1:
        return [0], []
    if n == 2:
        return [0], [1]

    # Fiedler vector
    _, eigenvecs = LA.eigsh(L, k=2, which='SA')   # 取最小两个特征值
    fiedler = eigenvecs[:, 1]
    order = np.argsort(fiedler)

    # 用 -L 的非对角元做邻接（拉普拉斯 L 的非对角是 -A）
    # 只用到严格上三角/下三角，不含对角度数项
    adj = -L[np.ix_(order, order)]

    ind = np.arange(1, n)
    # 质量：跨割边权 / (|S| * |S^c|)  —— 你的实现保持不变
    upper_tri_sums = np.array([np.sum(adj[i:, :i]) for i in ind])
    qualities = upper_tri_sums / (ind * (n - ind))
    best_cut = np.argmin(qualities) + 1

    g1 = order[:best_cut]
    g2 = order[best_cut:]
    return (g1, g2) if 0 in g1 else (g2, g1)


class BiCutNode:
    def __init__(self, indices, left=None, right=None):
        self.indices = indices
        self.left = left
        self.right = right

    def is_leaf(self):
        return self.left is None and self.right is None

    def get_order(self):
        order = []
        def dfs(node):
            if node.is_leaf():
                order.extend(node.indices)
            else:
                if node.left: dfs(node.left)
                if node.right: dfs(node.right)
        dfs(self)
        return order

    def print_fancy_tree(self, prefix="", is_last=True, is_root=True):
        if is_root:
            print("┌─ BiCut Tree Structure")
        connector = "├─" if is_root else ("└─" if is_last else "├─")
        print(f"{prefix}{connector} [{', '.join(map(str, sorted(self.indices)))}]")
        new_prefix = prefix + ("│  " if is_root else ("   " if is_last else "│  "))
        children = [c for c in (self.left, self.right) if c is not None]
        for i, child in enumerate(children):
            child.print_fancy_tree(new_prefix, i == len(children) - 1, False)

import os, math
from concurrent.futures import ThreadPoolExecutor

# === 直接使用你已有的 bicut_group 与 BiCutNode ===
# def bicut_group(L): ...（用你给的版本）
# class BiCutNode: ...（用你给的版本）

def treebuilder_parallel(L, thre=None, workers=None, max_parallel_depth=None):
    """
    简洁并行版 BiCut 构树（线程池）
    参数：
      - L: 整体拉普拉斯矩阵（numpy 或 scipy.sparse CSR/CSC）
      - thre: 叶子阈值（子块规模 ≤ thre 就不再切）
      - workers: 并行线程数（默认=CPU核数）
      - max_parallel_depth: 最大并行深度，超过就转串行，防止任务过多
    返回：
      - BiCutNode 根节点
    """
    if workers is None:
        workers = max(os.cpu_count() or 1, 1)

    # 经验：2^d >= workers ⇒ d ≈ ceil(log2(workers))
    if max_parallel_depth is None:
        max_parallel_depth = max(1, int(math.ceil(math.log2(workers))))

    # 内部递归函数：当 depth < max_parallel_depth 时并行左右子树，否则串行
    def _build(L_sub, indices, depth, executor):
        n = len(indices)
        # —— 基本情形：小块或阈值内就收叶子
        if n == 0:
            raise ValueError("矩阵为空。")
        if n == 1:
            return BiCutNode(indices)
        if n == 2:
            return BiCutNode(indices, BiCutNode([indices[0]]), BiCutNode([indices[1]]))
        if (thre is not None) and (n <= thre):
            return BiCutNode(indices)

        # —— 试着再切一刀
        g1_local, g2_local = bicut_group(L_sub)
        if len(g2_local) == 0:            # 切不动了，收叶子
            return BiCutNode(indices)

        # 局部→全局 索引
        g1 = [indices[i] for i in g1_local]
        g2 = [indices[i] for i in g2_local]

        # 子块拉普拉斯（与局部索引对齐）
        L11 = L_sub[np.ix_(g1_local, g1_local)]
        L22 = L_sub[np.ix_(g2_local, g2_local)]

        # —— 到这一步需要构左右子树：并行或串行
        if depth < max_parallel_depth and workers > 1 and executor is not None:
            # 并行提交左右子树
            f_left  = executor.submit(_build, L11, g1, depth + 1, executor)
            f_right = executor.submit(_build, L22, g2, depth + 1, executor)
            left  = f_left.result()
            right = f_right.result()
        else:
            # 超过并行深度，直接串行，避免任务过多
            left  = _build(L11, g1, depth + 1, executor)
            right = _build(L22, g2, depth + 1, executor)

        return BiCutNode(indices, left, right)

    # 线程池外层只开一次；把根任务丢进去递归构建
    with ThreadPoolExecutor(max_workers=workers) as ex:
        root = _build(L, list(range(L.shape[0])), 0, ex)
    return root


Defaulting to user installation because normal site-packages is not writeable


PY: C:\Users\xeangao\AppData\Local\Programs\Python\Python313\python.exe
VER: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
ENABLE_USER_SITE: True
USER_SITE: C:\Users\xeangao\AppData\Roaming\Python\Python313\site-packages
['C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\DLLs',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\Lib',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313',
 '',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\win32',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\xeangao\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\Pythonwin']


In [6]:
from tests import *

In [7]:
run_one_graph_test(1,20,20,20,False)

{'duration': 5.627856731414795,
 'memory': 52.73046016693115,
 'ratio': 1.0777642118214232,
 'matrix': array([[                 130, 18446744073709551615, 18446744073709551615,
         ...,                    0, 18446744073709551615,
                            0],
        [18446744073709551615,                  118,                    0,
         ..., 18446744073709551615, 18446744073709551615,
                            0],
        [18446744073709551615,                    0,                  123,
         ..., 18446744073709551615, 18446744073709551615,
                            0],
        ...,
        [                   0, 18446744073709551615, 18446744073709551615,
         ...,                  120, 18446744073709551615,
         18446744073709551615],
        [18446744073709551615, 18446744073709551615, 18446744073709551615,
         ..., 18446744073709551615,                  131,
         18446744073709551615],
        [                   0,                    0,        

In [8]:
run_one_graph_test(1,20,20,20,True)

{'duration': 5.801391124725342,
 'memory': 55.444167137145996,
 'ratio': 1.0708105257871743,
 'matrix': array([[                 130, 18446744073709551615, 18446744073709551615,
         ...,                    0, 18446744073709551615,
                            0],
        [18446744073709551615,                  118,                    0,
         ..., 18446744073709551615, 18446744073709551615,
                            0],
        [18446744073709551615,                    0,                  123,
         ..., 18446744073709551615, 18446744073709551615,
                            0],
        ...,
        [                   0, 18446744073709551615, 18446744073709551615,
         ...,                  120, 18446744073709551615,
         18446744073709551615],
        [18446744073709551615, 18446744073709551615, 18446744073709551615,
         ..., 18446744073709551615,                  131,
         18446744073709551615],
        [                   0,                    0,       

In [5]:
np.all(L1==L2)

np.True_

In [59]:
np.all(A==B)

np.True_

NameError: name 'eigsh' is not defined